<a href="https://colab.research.google.com/github/pratiksrm99/Abstractive_Text_Summerization/blob/main/Abstractive_TS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install pywsd
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from datasets import load_dataset
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from pywsd.lesk import adapted_lesk, simple_lesk, cosine_lesk
import pandas as pd
import numpy as np
import spacy
import re

Warming up PyWSD (takes ~10 secs)... took 9.80366826057434 secs.
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
dataset = load_dataset('gigaword')

  0%|          | 0/3 [00:00<?, ?it/s]

In [26]:
df = pd.DataFrame(columns= ["Document", "Summary"])
df["Document"] = dataset.get("test")["document"]
df["Summary"] = dataset.get("test")["summary"]

In [27]:
# df["Split_text"] = df["Document"].apply(str.split)
# df["Article_length"] = df["Split_text"].apply(len)

In [28]:
df.head()

,Document,Summary
0,japan 's nec corp. and UNK computer corp. of t...,nec UNK in computer sales tie-up
1,the sri lankan government on wednesday announc...,sri lanka closes schools as war escalates
2,police arrested five anti-nuclear protesters t...,protesters target french research ship
3,factory orders for manufactured goods rose #.#...,us september factory orders up #.# percent
4,the bank of japan appealed to financial market...,bank of UNK UNK for calm in financial markets


# **Implementation of Algorithm 1 from the paper**

Algorithm 1 Level-driven text generalization (LG)

Require: text, wsdText, F, T, θd
, θf

1: genText ← text

2: for all token ∈ text do

3: ftoken ← Frequency of token from F

4: if ftoken ≤ θf
then

5: c ← WSD of token form wsdText

6: Pc ← Path of hypernyms of c from T

7: dc ← taxonomy depth of concept c

8: fc ← Frequency of c from F

9: while fc < θf and dc > θd do

10: c ← hypernym of c from Ptoken

11: dc ← taxonomy depth of c

12: fc ← Frequency of c from F

13: end while

14: if (word of c) 6= token then

15: genText ← generalize token of genText to c

16: F ← (F \ {(token, ftoken)}) ∪ {(token, ftoken − 1)}

17: F ← (F \ {(c, fc
)}) ∪ {(c, fc + 1)}

18: end if

19: end if

20: end for

21: return genText

In [30]:
sent = "he is sitting on the bank of the river watching a speedboat in japan."
nlp = spacy.load("en_core_web_sm")

def one_func(x):
  try:
    if x.pos_=="NOUN":
      return adapted_lesk(sent,x.text).name()
    else:
      return x.text
  except:
    return x.text

def wsd_text(sent):
  doc = nlp(sent)
  test_df = pd.DataFrame(doc,columns=["text"])
  test_df["wsd"] = test_df["text"].apply(one_func)
  return " ".join(test_df["wsd"])

In [31]:
df["WSD Text"] = df["Document"].map(wsd_text)

In [32]:
df.head()

,Document,Summary,WSD Text
0,japan 's nec corp. and UNK computer corp. of t...,nec UNK in computer sales tie-up,japan 's nec corp . and UNK computer corp . of...
1,the sri lankan government on wednesday announc...,sri lanka closes schools as war escalates,the sri lankan politics.n.02 on wednesday anno...
2,police arrested five anti-nuclear protesters t...,protesters target french research ship,police.n.01 arrested five anti - nuclear disse...
3,factory orders for manufactured goods rose #.#...,us september factory orders up #.# percent,factory.n.01 regulate.v.02 for manufactured we...
4,the bank of japan appealed to financial market...,bank of UNK UNK for calm in financial markets,the bank of japan appealed to financial market...


In [33]:
a = df['WSD Text'].str.cat(sep=' ')
test_list = re.findall("[\w]+\..{1}\..{2}",a)
word_dist = FreqDist(test_list)
word_dist

FreqDist({'year.n.03': 193, 'state.n.04': 146, 'politics.n.02': 129, 'worldly_concern.n.01': 120, 'dollar.n.04': 108, 'official.s.05': 108, 'agency.n.01': 106, 'people.v.02': 105, 'sidereal_day.n.01': 94, 'submit.v.02': 78, ...})

In [34]:
thetha_d = 5
thetha_f = 100

def to_generalize(sent,thetha_d,thetha_f,word_dist):

  try:
    word_lst = [word for word in re.findall('[\w]+\..{1}\..{2}', sent) \
                if (word_dist[word] < thetha_f) and (len(wn.synset(word).hypernym_paths()[0])-1 > thetha_d)]

    for word in word_lst:
      while word_dist[word] < thetha_f and (len(wn.synset(word).hypernym_paths()[0])-1 > thetha_d):
        try:
          sent = sent.replace(word, str(wn.synset(word).hypernyms()[0].name()))
        except:
          sent = sent.replace(i, str(wn.synset(word).attributes()[0].name()))
        finally:
          word = wn.synset(word).hypernyms()[0].name()
  except:
    return "Error while generalizing"

  return sent

In [35]:
df["Generalized Text"] = df["WSD Text"].apply(lambda x: to_generalize(x,thetha_d,thetha_f,word_dist))
# xyz=23
# print(df["WSD Text"][xyz],"\n",df["Document"][xyz])

In [36]:
df[["Document","WSD Text", "Generalized Text"]]

,Document,WSD Text,Generalized Text
0,japan 's nec corp. and UNK computer corp. of t...,japan 's nec corp . and UNK computer corp . of...,japan 's nec corp . and UNK computer corp . of...
1,the sri lankan government on wednesday announc...,the sri lankan politics.n.02 on wednesday anno...,Error while generalizing
2,police arrested five anti-nuclear protesters t...,police.n.01 arrested five anti - nuclear disse...,force.n.04 arrested five anti - nuclear dissen...
3,factory orders for manufactured goods rose #.#...,factory.n.01 regulate.v.02 for manufactured we...,structure.n.01 regulate.v.02 for manufactured ...
4,the bank of japan appealed to financial market...,the bank of japan appealed to financial market...,the bank of japan appealed to financial struct...
...,...,...,...
1946,migrants should be regarded as a high-risk gro...,migrant.s.01 should be regarded as a high - ri...,migrant.s.01 should be regarded as a high - ri...
1947,the growth rate of indonesia 's manufacturing ...,the increase.n.03 rate.v.03 of indonesia 's ma...,the increase.n.03 rate.v.03 of indonesia 's ma...
1948,a mozambican man suspect of murdering jorge UN...,a mozambican world.n.08 suspect of murdering j...,a mozambican world.n.08 suspect of murdering j...
1949,the us dollar fell to the lower ### yen level ...,the us dollar.n.04 fell to the lower # # # yen...,the us dollar.n.04 fell to the lower # # # mon...


In [40]:
test_gen=to_generalize(wsd_text(sent),thetha_d,thetha_f,word_dist)
print(sent,"\n",test_gen)
wn.synset('instrumentality.n.03').definition()
df[df["Generalized Text"]!="Error while generalizing"]

he is sitting on the bank of the river watching a speedboat in japan. 
 he is sitting on the bank.n.01 of the river.n.01 watching a instrumentality.n.03 in japan .


,Document,Summary,WSD Text,Generalized Text
0,japan 's nec corp. and UNK computer corp. of t...,nec UNK in computer sales tie-up,japan 's nec corp . and UNK computer corp . of...,japan 's nec corp . and UNK computer corp . of...
2,police arrested five anti-nuclear protesters t...,protesters target french research ship,police.n.01 arrested five anti - nuclear disse...,force.n.04 arrested five anti - nuclear dissen...
3,factory orders for manufactured goods rose #.#...,us september factory orders up #.# percent,factory.n.01 regulate.v.02 for manufactured we...,structure.n.01 regulate.v.02 for manufactured ...
4,the bank of japan appealed to financial market...,bank of UNK UNK for calm in financial markets,the bank of japan appealed to financial market...,the bank of japan appealed to financial struct...
5,croatian president franjo tudjman said friday ...,rebel serb talks to resume saturday : tudjman ...,croatian president franjo tudjman said friday ...,croatian president franjo tudjman said friday ...
...,...,...,...,...
1946,migrants should be regarded as a high-risk gro...,migrant people need more aids control efforts ...,migrant.s.01 should be regarded as a high - ri...,migrant.s.01 should be regarded as a high - ri...
1947,the growth rate of indonesia 's manufacturing ...,major news items in leading vietnamese UNK,the increase.n.03 rate.v.03 of indonesia 's ma...,the increase.n.03 rate.v.03 of indonesia 's ma...
1948,a mozambican man suspect of murdering jorge UN...,mozambican suspected of killing UNK prison dir...,a mozambican world.n.08 suspect of murdering j...,a mozambican world.n.08 suspect of murdering j...
1949,the us dollar fell to the lower ### yen level ...,dollar falls to lower ### yen level in tokyo,the us dollar.n.04 fell to the lower # # # yen...,the us dollar.n.04 fell to the lower # # # mon...
